<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/08%20-%20Deep%20Learning%20e%20Chatbot/chatbot_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Powered Chatbot con Keras
In questo notebook creeremo un chatbot rudimentale in Python. Ho definito le conversazioni che il chatbot sarà in grado di gestire all'interno di un file JSON che puoi trovare sulla [repository github del corso](https://github.com/ProfAI/nlp00/blob/master/8%20-%20Chatbot/data/model.json).


### Carichiamo il corpus di testo
Utilizza pure il codice qui sotto per caricare il file JSON

In [1]:
import urllib.request, json 

with urllib.request.urlopen("https://raw.githubusercontent.com/ProfAI/nlp00/master/08%20-%20Deep%20Learning%20e%20Chatbot/data/model.json") as url:
    data = url.read().decode()
    
corpus = json.loads(data)
corpus

{'intents': [{'name': 'HelloIntent',
   'responses': ["Ciao, sono MiaoBot, l'assistente virtuale di Miao Mobile, come posso aiutarti ?",
    "Buongiorno, questa è l'assistenza clienti di Miao Mobile, come posso esserti utile ?"],
   'samples': ['Ciao', 'Salve', 'hei', 'Ci sei ?', 'Buongiorno', 'Buonasera']},
  {'name': 'GoodbyeIntent',
   'responses': ['Grazie per averci contattato ! Buona giornata',
    'Per qualsiasi cosa ci trovi sempre qui, buona giornata !',
    'Buona giornata !'],
   'samples': ['Addio',
    'Arrivederci',
    'Buona giornata',
    'A presto',
    'Ci vediamo']},
  {'name': 'ThanksIntent',
   'responses': ["E' stato un piacere, se hai bisogno di altro non esitare a chiedere",
    "E' stato un piacere aiutarti",
    'Figurati, è il mio lavoro :)'],
   'samples': ['Grazie', 'Ti ringrazio', 'Fantastico']},
  {'name': 'WhoIntent',
   'responses': ["Io sono MiaoBot, l'assistente virtuale di Miao Mobile, sono qui 24 ore su 24 per assisterti",
    "Mi chiamo MiaoBot, s

Il file contiene una serie di **intents** per un chatbot che simula un'operatore dell'assistenza clienti per una fantomatica compagnia telefonica chiamata Miao Mobile. Un'intent corrisponde ad un'azione che il chatbot può eseguire (in questo caso una risposta che può dare), ogni intent contiene questi elementi:
 - **name**: l'identificativo dell'intent.
 - **samples**: sono degli esempi di frasi che il chatbot deve imparare a riconoscere.
 - **responses**: sono le risposte che il chatbot deve fornire per l'intent corrispondente.
 

Questa codifica è basata su quella usata da Amazon Alexa, [qui trovi un'esempio](https://github.com/alexa/skill-sample-python-helloworld-decorators/blob/master/models/en-US.json). Il nostro obiettivo sarà quello di addestare una rete neurale usando le frasi di esempio per predire l'intent corrispondente, per poi usare l'intent per tornare una risposta presa a caso tra quelle disponibili.
 

### Preprocessiamo i dati
Cominciamo preprocessando i samples per ogni intent, per farlo ci serviremo di spacy, importiamo il modulo e, se non lo abbiamo già fatto, scarichiamo il modulo per la lingua italiana.

In [2]:
import spacy
!python -m spacy download it_core_news_sm

     |████████████████████████████████| 14.5 MB 8.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('it_core_news_sm')


Iteriamo su ogni sample di ogni intent ed eseguiamo queste trasformazioni sul testo:
 1. Converiamo tutto in minuscolo.
 2. Estraiamo i token.
 3. Verifichiamo che il token non sia un carattere di punteggiatura o una stop word.
 4. Estraiamo il lemma.
 5. Aggiungiamo il lemma al documento corrente.
 6. Se il lemma non è presenta all'interno del dizionario, aggiungiamolo.
 
 
 Per ogni documento salviamo anche il nome dell'intent corrispondente.

In [3]:
nlp = spacy.load("it_core_news_sm")

dictionary = set({})
intents = []

docs = []


for intent in corpus["intents"]:
  
  for sample in intent["samples"]:
    
    sample = sample.lower()
    tokens = nlp(sample)
    doc = ""
    
    for token in tokens:
      if(not token.is_punct and not token.is_stop):
        doc+=" "+token.lemma_
        dictionary.add(token.lemma_) # essendo un set, se il lemma è già presente non verrà aggiunto
        
    if(len(doc)>0):
      docs.append(doc.rstrip()) # usiamo rstrip() per rimuovere lo spazio alla fine del documento
      intents.append(intent["name"])
  
print("Lunghezza del dizionario: %d" % len(dictionary))
print(docs)
print(intents)

Lunghezza del dizionario: 57
[' ciao', ' salvo', ' hei', ' buongiorno', ' buonasera', ' addio', ' arrivederci', ' buono giornata', ' prestare', ' vedere', ' ringraziare', ' fantasticare', ' chiamare', ' parlare', ' potere sapere credito', ' qual essere credito', ' soldo', ' offrire', ' offrire', ' promozione', ' volere cambiare piare tariffario', ' volere cambiare promozione', ' volere una altro promozione', ' qual essere numerare verde', ' potere parlare una operatore', ' soluzione fibra', ' offrire lineare fisso', ' sentire', ' potere', ' potere aiutarmi', ' servire aiutare', ' bisognare assistenza', ' schifare', ' vaio farti friggere', ' totalmente inutile', ' umano', ' donna', ' una', ' donna', ' qual essere rispondere domandare fondamentale lo universo e', ' finire']
['HelloIntent', 'HelloIntent', 'HelloIntent', 'HelloIntent', 'HelloIntent', 'GoodbyeIntent', 'GoodbyeIntent', 'GoodbyeIntent', 'GoodbyeIntent', 'GoodbyeIntent', 'ThanksIntent', 'ThanksIntent', 'WhoIntent', 'WhoIntent'

### Bag of Words
Adesso codifichiamo il nostro testo utilizzando una rappresentazione bag of words, come al solito usiamo la classe *CountVectorizer* di sklearn.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
X = bow.fit_transform(docs)
X = X.toarray()
X.shape

(41, 56)

Gli intents sono i target della nostra rete neurale, al momento ogni intent è rappresentato da una stringa (l'identificativo), usiamo la classe *LabelEncoder* per codificarli in numeri.

In [30]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(intents)
y[:5]

array([8, 8, 8, 8, 8])

Eseguiamo il one hot encoding per creare le variabili di comodo per il target.
<br>
NOTA BENE Usando la classe OneHotEncoder di sklearn avremmo anche potuto evitare di eseguire la codifica dei label prima, dato che questa la effettua in automatico, il motivo per cui lo abbiamo fatto è che l'oggetto LabelEncoder che abbiamo creato ci servirà più avanti quando utilizzeremo la rete che abbiamo addestrato.

In [31]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(y.reshape(-1, 1))
y = y.toarray()
y.shape

(41, 16)

Mescoliamo il dataset usando la funzione shuffle di sklearn.

In [32]:
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=0)

Adesso siamo pronti per creare la nostra rete neurale artificiale.

## Creazione della rete
Creiamo la nostra archiettura di rete neurale artificiale usando Keras. Inizializziamo un modello sequenziale e aggiungiamo tre strati densi:
- Uno strato di input con 12 nodi.
- Uno strato nascosto con 8 nodi.
- Uno strato di output, con un numero di nodi pari al numero di tags da classificare.

In [33]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, activation="relu", input_dim=X.shape[1]))
model.add(Dense(8, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))

Compiliamo il modello, usando come come funzione di costo la *categorical crossentropy* e come algoritmo di ottimizzazione *adam*, aggiungiamo anche l'accuracy come metrica.

In [34]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

Avviamo l'addestramento con il metodo *.fit()*, impostando come numero di epoche 100.

In [36]:
model.fit(X, y, epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 5ms/step - loss: 2.7618 - accuracy: 0.0976
Epoch 2/500
2/2 [==============================] - 0s 5ms/step - loss: 2.7582 - accuracy: 0.0976
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7554 - accuracy: 0.1463
Epoch 4/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7526 - accuracy: 0.1463
Epoch 5/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7500 - accuracy: 0.1463
Epoch 6/500
2/2 [==============================] - 0s 6ms/step - loss: 2.7475 - accuracy: 0.1463
Epoch 7/500
2/2 [==============================] - 0s 4ms/step - loss: 2.7448 - accuracy: 0.1463
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7423 - accuracy: 0.1463
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7396 - accuracy: 0.1463
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 2.7371 - accuracy: 0.1707
Epoch 11/500
2/2 [===========

### Testiamo il chatbot
Adesso che la nostra rete è in grado di riconsocere l'intent di una richiesta, usiamola per creare il nostro chatbot. Definiamo una prima funzione che prende in ingresso la richiesta dell'utente e la processa esattamente come abbiamo processato i dati dell'addestramento.

In [ ]:
def preprocess(sentence):
  
  tokens = nlp(sentence.lower())
  doc = ""
  
  for token in tokens:
    if(not token.is_punct and not token.is_stop):
      doc+=" "+token.lemma_

  x = bow.transform([doc])
  
  return x

Definiamo una seconda funzione che prendendo in input un'intent ritorna la rispostsa del chatbot corrispondente a quell'intent. Per un'intent possono essere disponibili più risposte equivalenti, usiamo la funzione *choice* del modulo *random* per selezionare una delle risposte dalla lista a caso.

In [ ]:
from random import choice

def get_response(intent_name):
  
  for intent in corpus["intents"]:
    if(intent["name"]==intent_name):
      return choice(intent["responses"])

Adesso creiamo il core del chatbot, una funzione che prende in input la richiesta, la preprocessa, predice l'intent e ritorna la risposta.

In [ ]:
def chatbot(sentence):
  
  x = preprocess(sentence)
  y_proba = model.predict(x)[0]
  if(y_proba.max()>.7):
    y = y_proba.argmax()
    intent = le.inverse_transform([y])
    return get_response(intent)
  else:
    return "Temo proprio di non aver capito"

Ed adesso proviamo a chattare con il nostro chatbot (per chiudere la conversazione scriviamo 'arrivederci')

In [ ]:
sentence = ""

while(sentence.lower()!="arrivederci"):
  sentence = input("Tu: ")
  response = chatbot(sentence)
  print("Chatbot: "+response)

Tu: Arrivederci
Chatbot: Grazie per averci contattato ! Buona giornata


### Input multipli
L'input inserito dall'utente può anche contenere diverse richieste, usando *spacy* e l'attributo *.sents* possiamo dividerle e processarle singolarmente. 

In [ ]:
def preprocess(sentence):
  
  tokens = nlp(sentence)
  docs = []
  
  for sent in tokens.sents:
    
    doc = ""
    
    for token in sent:
      if(not token.is_punct and not token.is_stop):
        doc+=" "+token.lemma_
    
    docs.append(doc)
    
  X = bow.transform(docs)
  
  return X

Adesso la funzione response dovrà accettare una lista di tags e ritornare un'unica risposta unendo le risposte per ogni tag.

In [ ]:
from random import choice

def get_response(intents_name):
  
  response = ""
  
  for intent_name in intents_name:
  
    for intent in corpus["intents"]:
      if(intent["name"]==intent_name):
        response+=choice(intent["responses"])+" "
    
  return response

Proviamo di nuovo a chiacchierare con il nostro chatbot.

In [ ]:
sentence = None

while(sentence!="bye" and sentence!=""):
  sentence = input("Tu: ")
  response = chatbot(sentence)
  print("Chatbot: "+response)

Tu: Ciao amico mio ! Come ti senti oggi ?
Chatbot: Hei, felice di vederti Sto benone grazie ! 
Tu: bye
Chatbot: Ciao amico 
